In [4]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [5]:
import os
os.environ["OPENAI_API_KEY"] = ""

In [6]:
# provide the path of  pdf file/files.
pdfreader = PdfReader("budget_speech.pdf")

In [7]:
from typing_extensions import Concatenate
# read text from pdf
raw_text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        raw_text += content

In [9]:
# raw_text

In [10]:
# We need to split the text using Character Text Split such that it sshould not increse token size
text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 800,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [11]:
len(texts)

149

In [12]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [15]:
document_search = FAISS.from_texts(texts, embeddings)

In [16]:
document_search

In [17]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [18]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [19]:
query = "Vision for Amrit Kaal"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' Our vision for the Amrit Kaal includes technology-driven and knowledge-based economy with strong public finances, and a robust financial sector. To achieve this, Jan Bhagidari through Sabka Saath Sabka Prayas is essential. The economic agenda for achieving this vision focuses on three things: first, facilitating ample opportunities for citizens, especially the youth, to fulfil their aspirations; second, providing strong impetus to growth and job creation; and third, strengthening macro-economic stability.'

In [20]:
query = "How much the agriculture target will be increased to and what the focus will be"
docs = document_search.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The agriculture credit target will be increased to ` 20 lakh crore with focus on animal husbandry, dairy and fisheries.'